## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Erenhot,CN,43.6475,111.9767,79.57,28,19,8.28,few clouds
1,1,Pangnirtung,CA,66.1451,-65.7125,47.08,86,100,2.66,light rain
2,2,Atuona,PF,-9.8000,-139.0333,77.72,75,6,14.38,clear sky
3,3,Chalus,IR,36.6550,51.4204,78.26,78,75,0.00,broken clouds
4,4,Punta Arenas,CL,-53.1500,-70.9167,33.82,74,40,21.85,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
# Printing beginning to see
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Erenhot,CN,43.6475,111.9767,79.57,28,19,8.28,few clouds
2,2,Atuona,PF,-9.8000,-139.0333,77.72,75,6,14.38,clear sky
3,3,Chalus,IR,36.6550,51.4204,78.26,78,75,0.00,broken clouds
5,5,Hare Bay,CA,48.8499,-54.0148,63.75,94,100,13.49,overcast clouds
6,6,Kavieng,PG,-2.5744,150.7967,82.96,75,94,11.34,light rain
8,8,Geraldton,AU,-28.7667,114.6000,63.01,94,75,16.11,broken clouds
10,10,Rikitea,PF,-23.1203,-134.9692,73.44,74,0,3.85,clear sky
11,11,Ozinki,RU,51.1786,49.6769,64.58,56,14,7.47,few clouds
12,12,Thompson,CA,55.7435,-97.8558,62.76,82,75,14.97,broken clouds
13,13,Airai,TL,-8.9266,125.4092,63.37,80,89,3.76,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                437
City                   437
Country                436
Lat                    437
Lng                    437
Max Temp               437
Humidity               437
Cloudiness             437
Wind Speed             437
Weather Description    437
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
## 1 null row needs to be dropped.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

City_ID                436
City                   436
Country                436
Lat                    436
Lng                    436
Max Temp               436
Humidity               436
Cloudiness             436
Wind Speed             436
Weather Description    436
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Erenhot,CN,79.57,few clouds,43.6475,111.9767,
2,Atuona,PF,77.72,clear sky,-9.8000,-139.0333,
3,Chalus,IR,78.26,broken clouds,36.6550,51.4204,
5,Hare Bay,CA,63.75,overcast clouds,48.8499,-54.0148,
6,Kavieng,PG,82.96,light rain,-2.5744,150.7967,
8,Geraldton,AU,63.01,broken clouds,-28.7667,114.6000,
10,Rikitea,PF,73.44,clear sky,-23.1203,-134.9692,
11,Ozinki,RU,64.58,few clouds,51.1786,49.6769,
12,Thompson,CA,62.76,broken clouds,55.7435,-97.8558,
13,Airai,TL,63.37,overcast clouds,-8.9266,125.4092,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Erenhot,CN,79.57,few clouds,43.6475,111.9767,Erenhot Rurouni Youth Hostel
2,Atuona,PF,77.72,clear sky,-9.8000,-139.0333,Villa Enata
3,Chalus,IR,78.26,broken clouds,36.6550,51.4204,Rama Spa
5,Hare Bay,CA,63.75,overcast clouds,48.8499,-54.0148,
6,Kavieng,PG,82.96,light rain,-2.5744,150.7967,Nusa Island Retreat
8,Geraldton,AU,63.01,broken clouds,-28.7667,114.6000,Broadwater Mariner Resort
10,Rikitea,PF,73.44,clear sky,-23.1203,-134.9692,People ThankYou
11,Ozinki,RU,64.58,few clouds,51.1786,49.6769,Korolevskiy Dvor
12,Thompson,CA,62.76,broken clouds,55.7435,-97.8558,Burntwood Hotel
13,Airai,TL,63.37,overcast clouds,-8.9266,125.4092,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df.loc[(hotel_df["Hotel Name"] == "")] = np.nan

clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Erenhot,CN,79.57,few clouds,43.6475,111.9767,Erenhot Rurouni Youth Hostel
2,Atuona,PF,77.72,clear sky,-9.8000,-139.0333,Villa Enata
3,Chalus,IR,78.26,broken clouds,36.6550,51.4204,Rama Spa
6,Kavieng,PG,82.96,light rain,-2.5744,150.7967,Nusa Island Retreat
8,Geraldton,AU,63.01,broken clouds,-28.7667,114.6000,Broadwater Mariner Resort
10,Rikitea,PF,73.44,clear sky,-23.1203,-134.9692,People ThankYou
11,Ozinki,RU,64.58,few clouds,51.1786,49.6769,Korolevskiy Dvor
12,Thompson,CA,62.76,broken clouds,55.7435,-97.8558,Burntwood Hotel
15,Vaini,TO,78.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Saint-Leu,RE,67.06,scattered clouds,-21.1500,55.2833,La Caz Oceane


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))